# St Louis Parcels Nearby to Applicants

List all parcels within 500 meters of each applicants' parcel(s).

Import packages and set working directory.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

os.chdir(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook')

Load output from "St. Louis Address Matching" and reshape the data from wide to long. (Some applications are associated with more than one parcel.)

In [2]:
# Load matches
matched = pd.read_csv('final_handle_assignments.csv')
# Reshape wide to long, putting all applicant parcels in a single column
matched_long = pd.melt(matched, id_vars=['authorized', 'best_addr1', 'best_zip', 'issued', 'owner', 'refnum', 'result', 'year', 'match_type'], value_vars=['HANDLE_M1', 'HANDLE_M2', 'HANDLE_M3', 'HANDLE_M4'])
# Drop rows with no parcel (created when applications have less than 4 parcels)
matched_long = matched_long[matched_long['value'].notnull()]

Cut down to an unduplicated number of applicant parcels, for which to find nearby parcels

In [3]:
uniq_applicant_parcels = matched_long[['value']].drop_duplicates()
uniq_applicant_parcels = uniq_applicant_parcels.rename(columns={'value': 'HANDLE'})

Load the shapefile of St. Louis parcels, from the Assessor's website. Cut it down to the subset of unique applicant parcels, by merging. Save as a new shapefile.

In [4]:
prcl = gpd.read_file('shapefiles\\prcl.shp')
prcl['HANDLE'] = prcl['HANDLE'].astype(np.float64)
prcl_applicants = prcl.merge(uniq_applicant_parcels, left_on='HANDLE', right_on='HANDLE', how='inner')

prcl_applicants.to_file('shapefiles\\prcl_applicants.shp')

**The next cell requires an ArcPy environment.** Either import ARCPY in an ArcGIS-compatible installation of Python or (easier) paste into the Python window of ArcGIS.

Load the two shapefiles with (A) applicants' parcels and (B) all of St. Louis. Find all parcels in B that are within 500 meters of each parcel in A.

Note that this takes a while, so only re-run if necessary.

In [ ]:
# Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True

# Load shapefiles
    #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\prcl_applicants.shp', 'prcl_applicants', None, None, None, None)
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\prcl.shp', 'prcl', None, None, None, None)
# Find nearby parcels
    #GenerateNearTable_analysis (in_features, near_features, out_table, {search_radius}, {location}, {angle}, {closest}, {closest_count}, {method})
arcpy.analysis.GenerateNearTable("prcl_applicants", "prcl", r"C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb\prcl_applicants_nearTable", "500 Meters", "NO_LOCATION", "NO_ANGLE", "ALL", 10000, "PLANAR")

# Export to a text file. This only includes ArcGIS index numbers for the parcels, not their handles.
arcpy.management.CopyRows(r"C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb\prcl_applicants_nearTable", r"C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\nearTable1.txt", None)

Import the ArcGIS output back to a Pandas dataframe and clean it up. 

In [5]:
#Load
nearTable = pd.read_csv('nearTable1.txt')
# ArcGIS indices start from 1. Adjust them to match Python's indices.
nearTable['IN_FID'] = nearTable['IN_FID']-1
nearTable['NEAR_FID'] = nearTable['NEAR_FID']-1

# prcl is the (geo)dataframe for all of St Louis. Rename the dataframe and column name.
prcls_near = prcl[['HANDLE']]
prcls_near['HANDLE_NEAR'] = prcls_near['HANDLE']

# Merge the parcel handles onto the parcel indices outputted by ArcGIS
nearTable = nearTable.merge(prcl_applicants[['HANDLE']],left_on='IN_FID', right_index=True, how='left')
nearTable = nearTable.merge(prcls_near[['HANDLE_NEAR']],left_on='NEAR_FID', right_index=True, how='left')

# Convert the distance measure from feet to meters
nearTable['NEAR_DIST_METERS'] = nearTable['NEAR_DIST'] * 0.3048
# Cut to relevant columns
nearTable = nearTable[['NEAR_DIST_METERS', 'NEAR_RANK', 'HANDLE', 'HANDLE_NEAR']]

Merge to the "long" list of applications, which includes duplicate parcels and has all parcel number in one column. 

Keep only one distance per application and nearby parcel. Pick the shortest one. (ie, If one application is associated with two parcels, there are two distances from that application to each nearby parcel.)

In [6]:
#Merge
matched_long = matched_long.merge(nearTable, left_on='value', right_on='HANDLE', how='left')
# Sort to put shortest distance first
matched_long = matched_long.sort_values(['refnum', 'result', 'NEAR_DIST_METERS'], axis=0)
# Keep one distance for each application (['refnum', 'result'] form a joint unique ID)
matched_long = matched_long.drop_duplicates(['refnum', 'result', 'HANDLE_NEAR'])
# Cut to relevant columns
matched_long = matched_long[['refnum', 'result', 'NEAR_RANK', 'HANDLE_NEAR', 'NEAR_DIST_METERS']]

This currently includes distance from applicant parcels to themselves. Merge to the original (wide) list of applicant parcels and eliminate distance from applications' parcels to themselves.

In [7]:
matched_long = matched_long.merge(matched, left_on=['refnum', 'result'], right_on=['refnum', 'result'], how='left')

matched_long = matched_long[matched_long['HANDLE_NEAR']!=matched_long['HANDLE_M1']]
matched_long = matched_long[matched_long['HANDLE_NEAR']!=matched_long['HANDLE_M2']]
matched_long = matched_long[matched_long['HANDLE_NEAR']!=matched_long['HANDLE_M3']]
matched_long = matched_long[matched_long['HANDLE_NEAR']!=matched_long['HANDLE_M4']]

# Cut to relevant columns
matched_long = matched_long[['refnum', 'result', 'NEAR_RANK', 'HANDLE_NEAR', 'NEAR_DIST_METERS']]

Export to CSV.

In [ ]:
matched_long.to_csv('nearby_parcels.csv', index=False)

**Demonstrate results:** Nearby parcels for the application with Refnum=1 and result=Denied, spreadsheet and map.
<img src="NearbyParcelsRefnum1Denied_spreadsheet.PNG">
<img src="NearbyParcelsRefnum1Denied.PNG">